# Practica 3: Jaime Héctor y Álvaro Sánchez

## Preprocesamiento de datos

En primer lugar, es necesario realizar un preprocesamiento de los datos para garantizar que estén limpios, completos y en un formato adecuado que permita al modelo de machine learning aprender de manera eficiente. Para ello se deben evitar los valores nulos, normalizar las variables numéricas y codificar las variables categóricas.

In [15]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Cargar los datos en una variable
datos_test = pd.read_csv("public_test.csv")  # Fichero CSV con datos de test
datos_train = pd.read_csv("public_train.csv")  # Fichero CSV con datos de entrenamiento

# Separar la información del resultado ovariable objetivo
x_train = datos_train.drop(columns="ccs")  # 'ccs' es la columna objetivo
y_train = datos_train["ccs"]
x_test = datos_test

# Iniciar transmormadores para adaptar los datos al modelo
scaler = StandardScaler()
numeric_imputer = SimpleImputer(strategy="mean")  # Nulo numérico --> Media del resto
categorical_imputer = SimpleImputer(strategy="most_frequent")  # Nulo categórico --> Moda del resto
oh_encoder = OneHotEncoder(sparse_output=False)  # One-hot encoding: Codificar variables categóricas

# Transformador numérico (Eliminar nulos + normalizar)
numeric_transformer = Pipeline(steps=[
    ("imputer", numeric_imputer),
    ("scaler", scaler)
])

# Transformador categórico (Eliminar nulos + codificar)
categorical_transformer = Pipeline(steps=[
    ("imputer", categorical_imputer),
    ("encoder", oh_encoder)
])

# Agrupar las diferentes columnas relevantes de los datos
desc_columns = [col for col in x_train.columns if col.startswith('desc_')]
fgp_columns = [col for col in x_train.columns if col.startswith('fgp_')]
adduct_column = ["adduct"]  # 'adduct' es categórica 

# Transformar usando ColumnTransformer, para elegir a que columnas aplicar cada tipo de transformación
transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, desc_columns + fgp_columns),  # Numéricos
        ("categorical", categorical_transformer, adduct_column)  # Categóricos
    ]
)


# Aplicar los cambios realizados en los datos (no los resultados)

x_train = transformer.fit_transform(x_train)
x_test = transformer.transform(x_test)


## Entrenamiento y estimación del error

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import median_absolute_error

# Dividir los datos: 70% entrenamiento, 20% validación
x_train_final, x_val, y_train_final, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

model1 = LinearRegression()

model1.fit(x_train_final, y_train_final)

y_val_pred1 = model1.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred1)
    
print(f"Para Regresión lineal se obtiene una mediana de error: {mae_val}")

Para Regresión lineal se obtiene una mediana de error: 3.7602733080887845


In [28]:
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(5)

model2.fit(x_train_final, y_train_final)

y_val_pred2 = model2.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred2)
    
print(f"Para Random Forest con 5 estimadores se obtiene una mediana de error: {mae_val}")


Para Random Forest con 5 estimadores se obtiene una mediana de error: 3.3062499999999915


In [27]:
from sklearn.ensemble import RandomForestRegressor

model3 = RandomForestRegressor(10)

model3.fit(x_train_final, y_train_final)

y_val_pred3 = model3.predict(x_val)
mae_val = median_absolute_error(y_val, y_val_pred3)
    
print(f"Para Random Forest con 10 estimadores se obtiene una mediana de error: {mae_val}")

Para Random Forest con 10 estimadores se obtiene una mediana de error: 3.2440015000000244


## Generación de predicciones

## Conclusiones